Sheet 3.3: Prompting & Decoding
=======
**Author**: Polina Tsvilodub & Michael Franke

This sheet provides more details on concepts that have been mentioned in passing in the previous sheets, and provides some practical examples and exercises for prompting techniques that have been covered in lecture four. Therefore, the learning goals for this sheet are:
* take a closer look and understand various decoding schemes,
* understand the **temperature** parameter,
* see a few practical examples of prompting techniques from the lecture.

## Decoding schemes

This part of this sheet is a close replication of [this](https://michael-franke.github.io/npNLG/06-LSTMs/06d-decoding-GPT2.html) sheet.

This topic addresses the following question: Given a language model that outputs a next-word probability, how do we use this to actually generate naturally sounding text? For that, we need to choose a single next token from the distribution, which we will then feed back to the model, together with the preceding tokens, so that it can generate the next one. This inference procedure is repeated, until the EOS token is chosen, or a maximal sequence length is achieved. **The procedure of how exactly to get that single token from the distribution is call *decoding scheme*.** Note that "decoding schemes" and "decoding strategies" refer to the same concept and are used interchangeably.

We have already discussed decoding schemes in lecture 02 (slide 25). The following introduces these schemes in more detail again and provides example code for configuring some of them.

> <strong><span style=&ldquo;color:#D83D2B;&rdquo;>Exercise 3.3.1: Decoding schemes</span></strong>
>
> Please read through the following introduction and look at the provided code.
> 1. With the help of the example and the documentation, please complete the code (where it says "### YOUR CODE HERE ####") for all the decoding schemes.

Common decoding strategies are:
* **pure sampling**: In a pure sampling approach, we just sample each next word with exactly the probability assigned to it by the LM. Notice that this process, therefore, is **non-determinisitic**. We can force replicable results, though, by setting a *seed*.
* **Softmax sampling**: In soft-max sampling, the probablity of sampling word $w_i$ is $P_{LM} (w_i \mid w_{1:i-1}) \propto \exp(\frac{1}{\tau} P_{LM}(w_i \mid w_{1:i-1}))$, where $\tau$ is a *temperature parameter*.
  * The *temperature parameter* is also often available for closed-source models like the GPT family. It is often said to change the "creativity" of the output.
* **greedy sampling**: In greedy sampling, **we don’t actually sample but just take the most likely next-word** at every step. Greedy sampling is equivalent to setting $\tau = 0$ for soft-max sampling. It is also sometimes referred to as *argmax* decoding.
* **beam search**: In simplified terms, beam search is a parallel search procedure that keeps a number $k$ of path probabilities open at each choice point, dropping the least likely as we go along. (There is actually **no unanimity** in what exactly beam search means for NLG.)
* **top-$k$ sampling**: his sampling scheme looks at the $k$ most likely next-words and samples from so that: $$P_{\text{sample}}(w_i  \mid w_{1:i-1}) \propto \begin{cases} P_{M}(w_i \mid w_{1:i-1}) & \text{if} \; w_i \text{ in top-}k \\ 0 & \text{otherwise} \end{cases}$$
* **top-$p$ sampling**: Top-$p$ sampling is similar to top-$k$ sampling, but restricts sampling not to the top-$k$ most likely words (so always the same number of words), but the set of most likely words the summed probability of which does not exceed threshold $p$.

The within the `transformers` package, **for all causal LMs**, the `.generate()` function is available which allows to sample text from the model (remember the brief introduction in [sheet 2.5](https://cogsciprag.github.io/Understanding-LLMs-course/tutorials/02e-intro-to-hf.html)). Configuring this function via different values and combinations of various parameters allows to sample text with the different decoding schemes described above. The respective documentation can be found [here](https://huggingface.co/docs/transformers/v4.40.2/en/generation_strategies#decoding-strategies). The same configurations can be passed to the `pipeline` endpoint which we have seen in the same sheet.

Check out [this](https://medium.com/@harshit158/softmax-temperature-5492e4007f71) blog post for very noce visualizations and more detials on the *temperature* parameter.

Please complete the code below. GPT-2 is used as an example model, but this works exactly the same with any other causal LM from HF.

In [1]:
!pip install accelerate
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
# import relevant packages
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
model = GPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

# convenience function for nicer output
def pretty_print(s):
    print("Output:\n" + 100 * '-')
    print(tokenizer.decode(s, skip_special_tokens=True))

# encode context the generation is conditioned on
input_ids = tokenizer.encode('I enjoy walking with my cute dog', return_tensors='pt')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [3]:
# model.generate??

In [4]:
# set a seed for reproducibility (if you want)
torch.manual_seed(199)

MAX_LENGTH = 50
MAX_NEW_TOKENS = 20


# below, greedy decoding is implemented
# NOTE: while it is the default for .generate(), it is NOT for pipeline()
print('\ngreedy decoding')
greedy_output = model.generate(input_ids, max_new_tokens=MAX_NEW_TOKENS)
print(pretty_print(greedy_output[0]))

# here, beam search is shown
# option `early_stopping` implies stopping when all beams reach the end-of-sentence token
beam_output = model.generate(
    input_ids,
    max_new_tokens=MAX_NEW_TOKENS,
    num_beams=3,
    early_stopping=True
)
print('\nbeam search decoding')
pretty_print(beam_output[0])


#  pure sampling
sample_output = model.generate(
    input_ids,        # context to continue
    #### YOUR CODE HERE ####
    do_sample=True, # True enables strategies: multinomial sampling, beam-search multinomial sampling, Top-K sampling and Top-p sampling.
    max_new_tokens=MAX_NEW_TOKENS, # return maximally 10 new tokens (following the input)
)
print('\npure sampling decoding')
pretty_print(sample_output[0])

# same as pure sampling before but with `temperature`` parameter
SM_sample_output = model.generate(
    input_ids,  # context to continue
    #### YOUR CODE HERE ####
    do_sample=True,
    # temperature=0.0001,  # very small temperature corresponds to greedy decoding
    temperature=0.9,
    max_new_tokens=MAX_NEW_TOKENS,
)

print('\npure sampling with temperature')
pretty_print(SM_sample_output[0])

# top-k sampling
top_k_output = model.generate(
    input_ids,
    ### YOUR CODE HERE ####
    do_sample=True,
    top_k=50,
    max_new_tokens=MAX_NEW_TOKENS,
)
print('\ntop-k sampling')
pretty_print(top_k_output[0])

# top-p sampling
top_p_output = model.generate(
    input_ids,
    ### YOUR CODE HERE ####
    do_sample=True,
    top_p=0.90, # the size of the set of words (V) can dynamically increase (or dec.) according to the next word's probability distribution.
    max_new_tokens=MAX_NEW_TOKENS,
    # max_length=MAX_LENGTH,
)
print('\ntop-p sampling')
pretty_print(top_p_output[0])


greedy decoding
Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with my dog. I'm
None

beam search decoding
Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I don't think I'll ever be able to walk with my dog again.



pure sampling decoding
Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog. It lets me know that I'm not alone and can make a difference for others. I get

pure sampling with temperature
Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, Chubby. We often go to the grocery or the park, to pick up food for our

top-k sampling
Output:
-------------------------

> <strong><span style=&ldquo;color:#D83D2B;&rdquo;>Exercise 3.3.2: Understanding decoding schemes</span></strong>
>
> Think about the following questions about the different decoding schemes.
>  
> 1. Why is the temperature parameter in softmax sampling sometimes referred to as a creativity parameter? Hint: Think about the shape distribution and from which the next word is sampled, and how it compares to the "pure" distribution when the temperature parameter is varied.
> 2. Just for yourself, draw a diagram of how beam decoding that starts with the BOS token and results in the sentence "BOS Attention is all you need" might work, assuming k=3 and random other tokens of your choice.
> 3. Which decoding scheme seems to work best for GPT-2?

        - beam search and top-k sampling as they give reasonable and complete sentences?

> 4. Which of the decoding schemes included in this work sheet is a special case of which other decoding scheme(s)? E.g., X is a special case of Y if the behavior of Y is obtained when we set certain paramters of X to specific values.
> 5. Can you see pros and cons to using some of these schemes over others?

**Outlook**

There are also other more recent schemes, e.g., [locally typical sampling](https://arxiv.org/abs/2202.00666) introduced by Meister et al. (2022).

## Prompting strategies

The lecture introduced different prompting techniques. (Note: "prompting technique" and "prompting strategy" refer to the same concept and are used interchangeably)
Prompting techniques refer to the way (one could almost say -- the art) of constructing the inputs to the LM, so as to get optimal outputs for your task at hand. **Note that prompting is complementary to choosing the right decoding scheme** -- one still has to choose the decoding scheme for predicting the completion, given the prompt constructed via a particulat prompting strategy.

Below, a practical example of a simple prompting strategy, namely *few-shot prompting* (which is said to **elicit** *in-context learning*), and a more advanced example, namely *generated knowledge prompting* are provided. These should serve as inspiration for your own implementations and explorations of other prompting schemes out there. Also, feel free to play around with the examples below to build your intuitions! Of course, you can also try different models, sentences, ...

**Note**

You might have already experienced rate limits of accessing the GPU on Colab. To try to avoid difficulties with completing the tasks on GPU, if you want to use Colab, here are a few potential aspects (approximated by experience, definitely non-exhaustive and inofficial) that might lead to rate limits: requesting GPU runtimes and then not utilizing the GPU, requesting a lot of GPU runtimes (e.g., multiple per day), running very long jobs (multiple hours).
To try to work around this, one possibility is to debug and test code that doesn't require GPUs in non-GPU runtimes, and only request those when actually needed.

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd
import numpy as np

In [6]:
# define computational device
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Device: {device}")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"Device: {device}")
else:
    device = torch.device("cpu")
    print(f"Device: {device}")

Device: cuda


In [21]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/Pythia-1.4b")  # pythia-1.4b is also part of hw2
# pythia was not instruction finetuned, so it might have a hard time following structured texts.
# try by making the prompt a paragraph and see if it follows.

model = AutoModelForCausalLM.from_pretrained(
    "EleutherAI/Pythia-1.4b",
    # trust_remote_code=True,
    # torch_dtype=torch.float16,
).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [65]:
# few shot prompting

few_shot_prompt = """
Input: This class is awesome. Sentiment: positive
Input: This class is terrible. Sentiment: neutral
Input: The class is informative. Sentiment: neutral
"""
input_text = "The class is my favourite!"

full_prompt = few_shot_prompt + "Input: " + input_text + " Sentiment: "

input_ids = tokenizer(full_prompt, return_tensors="pt").input_ids.to(device)
few_shot_prediction = model.generate(
    input_ids,
    max_new_tokens=MAX_NEW_TOKENS,


    # do_sample=True,
    # temperature=0.9,

    # num_beams=10,
    # early_stopping=True,

    # do_sample=True,
    # top_k=3,

    # do_sample=True,
    # top_p=0.90,

)

print(tokenizer.decode(few_shot_prediction[0], skip_special_tokens=False))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Input: This class is awesome. Sentiment: positive
Input: This class is terrible. Sentiment: neutral
Input: The class is informative. Sentiment: neutral
Input: The class is my favourite! Sentiment: 
Input: The class is my least favourite. Sentiment: negative
Input: The class is


In [75]:
# few shot prompting

few_shot_prompt = """
Consider the sentence 'This class is awesome'. It means whoever said this was happy with the class and so we label this sentence
as of positive sentiment.
Consider the sentence 'This class is terrible'. It means whoever said this was not happy with the class and so we label this sentence
as of negative sentiment.
Consider the sentence 'The class is informative'. It means whoever said this did not have strong opinions about the class and so we label this sentence
as of neutral sentiment.
"""
input_text = "Consider the sentence 'The class is my favourite!' "

full_prompt = few_shot_prompt + input_text

input_ids = tokenizer(full_prompt, return_tensors="pt").input_ids.to(device)
few_shot_prediction = model.generate(
    input_ids,
    max_new_tokens=25,


    # do_sample=True,
    # temperature=0.9,

    # num_beams=10,
    # early_stopping=True,

    # do_sample=True,
    # top_k=3,

    # do_sample=True,
    # top_p=0.90,

)

print(tokenizer.decode(few_shot_prediction[0], skip_special_tokens=False))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Consider the sentence 'This class is awesome'. It means whoever said this was happy with the class and so we label this sentence
as of positive sentiment.
Consider the sentence 'This class is terrible'. It means whoever said this was not happy with the class and so we label this sentence
as of negative sentiment.
Consider the sentence 'The class is informative'. It means whoever said this did not have strong opinions about the class and so we label this sentence
as of neutral sentiment.
Consider the sentence 'The class is my favourite!' 
It means whoever said this was happy with the class and so we label this sentence as of positive sentiment.
Consider the


In [76]:
# few shot prompting

few_shot_prompt = """
Consider the sentence 'This class is awesome'. It means whoever said this was happy with the class and so we label this sentence
as of positive sentiment.
Consider the sentence 'This class is terrible'. It means whoever said this was not happy with the class and so we label this sentence
as of negative sentiment.
Consider the sentence 'The class is informative'. It means whoever said this did not have strong opinions about the class and so we label this sentence
as of neutral sentiment.
"""
input_text = "Consider the sentence 'The class is my least favourite!' "

full_prompt = few_shot_prompt + input_text

input_ids = tokenizer(full_prompt, return_tensors="pt").input_ids.to(device)
few_shot_prediction = model.generate(
    input_ids,
    max_new_tokens=25,


    # do_sample=True,
    # temperature=0.9,

    # num_beams=10,
    # early_stopping=True,

    # do_sample=True,
    # top_k=3,

    # do_sample=True,
    # top_p=0.90,

)

print(tokenizer.decode(few_shot_prediction[0], skip_special_tokens=False))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Consider the sentence 'This class is awesome'. It means whoever said this was happy with the class and so we label this sentence
as of positive sentiment.
Consider the sentence 'This class is terrible'. It means whoever said this was not happy with the class and so we label this sentence
as of negative sentiment.
Consider the sentence 'The class is informative'. It means whoever said this did not have strong opinions about the class and so we label this sentence
as of neutral sentiment.
Consider the sentence 'The class is my least favourite!' 
It means whoever said this was not happy with the class and so we label this sentence as of negative sentiment.
Consider


In [56]:
print(full_prompt)


Input: This class is awesome. Sentiment: positive
Input: This class is terrible. Sentiment: neutral
Input: The class is informative. Sentiment: neutral
Input: The class is my favourite! Sentiment: 


Example of generated knowledge prompting (somewhat approximated, based on code from [this class](https://cogsciprag.github.io/LLM-implications/materials/session5)), as introduced by [Liu et al. (2022)](https://aclanthology.org/2022.acl-long.225.pdf).
This prompting technique is used to answer this multiple-choice question from the CommonsenseQA benchmark: "Where would you expect to find a pizzeria while shopping?". The answer options are: A = ["chicago", "street", "little italy", "food court", "capital cities"]

As a reminder, the overall idea of generated knowledge prompting is the following:
* knowledge generation: given question $Q$ and a few-shot example, generate a set $K_Q$ of $k$ knowledge statements
  * we will load the few-shot examples from a csv file [here](https://cogsciprag.github.io/Understanding-LLMs-course/tutorials/files/knowledge_examples.csv).
* knowledge integration: given $Q$ and $K_Q$, retrieve the log probabilities of each answer option $a_i \in A$ and select the option with the highest probability.
  * in the paper, this is done separately for each knowledge statement in $K_Q$. As a simplification, we will concatenate all $K_Q$ and compare the answer options given this combined prompt.

In [57]:
# 1. construct few-shot example

question = "Where would you expect to find a pizzeria while shopping?"
answers = ["chicago", "street", "little italy", "food court", "capital cities"]

examples_df = pd.read_csv("knowledge_examples.csv", sep = "|")

few_shot_template = """{q} We know that {k}"""

few_shot_prompt = "\n".join([
    few_shot_template.format(
        q=examples_df.loc[i, "input"],
        k=examples_df.loc[i, "knowledge"].lower()
    )
    for i in range(len(examples_df))
])
print("Constructed few shot prompt\n", few_shot_prompt)

Constructed few shot prompt
 How many wings do penguins have? We know that birds have two wings. penguin is a kind of bird.
WHat is the number of limbs a typical human being has? We know that human beings have four limbs.


In [58]:
examples_df

,Unnamed: 0,input,knowledge
0,0,How many wings do penguins have?,Birds have two wings. Penguin is a kind of bird.
1,1,WHat is the number of limbs a typical human be...,Human beings have four limbs.


In [60]:
# 2. generate knowledge statements
# tokenize few shot prompt together with our actual question
prompt_input_ids = tokenizer(
    few_shot_prompt + "\n" + question + " We know that ",
    return_tensors="pt"
).input_ids.to(device)

knowledge_statements = model.generate(
    prompt_input_ids,
    max_new_tokens=10,
    do_sample=True,
    temperature=0.4
)
# access the knowledge statements (i.e., only text that comes after prompt)
knowledge = tokenizer.decode(
    knowledge_statements[0, prompt_input_ids.shape[-1]:],
    skip_special_tokens=True
)

print(tokenizer.decode(knowledge_statements[0]))
print("\nGenerated knowledge:\n", knowledge)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


How many wings do penguins have? We know that birds have two wings. penguin is a kind of bird.
WHat is the number of limbs a typical human being has? We know that human beings have four limbs.
Where would you expect to find a pizzeria while shopping? We know that 
pizza is a kind of food.


Generated knowledge:
 
pizza is a kind of food.



In [61]:
prompt

'\na pizzeria is a place where people eat pizza.\nWhat Where would you expect to find a pizzeria while shopping? capital cities'

In [68]:
# 3. Score each answer to the question based on the knowledge statements
# as the score, we take the average log probability of the tokens in the answer

answer_log_probs = []
# iterate over the answer options
# NOTE: This can take a moment
for a in answers:
    # construct the full prompt
    prompt = f"{knowledge} {question} {a}" # note: so here we are already giving the model the answer, and observing the loss!
    # construct the prompt without the answer to create a mask which will
    # allow to retrieve the token probabilities for tokens in the answer only
    context_prompt = f"{knowledge} {question}"
    # tokenize the prompt
    input_ids = tokenizer(prompt,
                          return_tensors="pt").input_ids.to(device)
    # tokenize the context prompt
    context_input_ids = tokenizer(context_prompt,
                                  return_tensors="pt").input_ids
    # create a mask with -100 for all tokens in the context prompt
    # the -100 indicates that the token should be ignored in the loss computation
    masked_labels = torch.ones_like(input_ids) * -100
    masked_labels[:, context_input_ids.shape[-1]:] = input_ids[:, context_input_ids.shape[-1]:]
    print("Mask ", masked_labels)
    # generate the answer
    preds = model(
        input_ids,
        labels=masked_labels
    )
    # retrieve the average log probability of the tokens in the answer
    log_p = preds.loss.detach().cpu().item()
    answer_log_probs.append(-log_p)
    print("Answer ", a, "Average log P ", log_p)

Mask  tensor([[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,  448,
         7298]], device='cuda:0')
Answer  chicago Average log P  7.39178466796875
Mask  tensor([[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 6406]],
       device='cuda:0')
Answer  street Average log P  13.069877624511719
Mask  tensor([[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 1652,
          352, 5242]], device='cuda:0')
Answer  little italy Average log P  7.875202178955078
Mask  tensor([[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 2739,
         1302]], device='cuda:0')
Answer  food court Average log P  8.3958177

In [69]:
# 4. retrieve the answer option with the highest score
# find max probability
print("All answers ", answers)
print("Answer probabilities ", answer_log_probs)
max_prob_idx = np.argmax(answer_log_probs)
print("Selected answer ", answers[max_prob_idx], "with log P ", answer_log_probs[max_prob_idx])

All answers  ['chicago', 'street', 'little italy', 'food court', 'capital cities']
Answer probabilities  [-7.39178466796875, -13.069877624511719, -7.875202178955078, -8.395817756652832, -10.538715362548828]
Selected answer  chicago with log P  -7.39178466796875


> <strong><span style=&ldquo;color:#D83D2B;&rdquo;>Exercise 3.3.3: Prompting techniques</span></strong>
>
> For the following exercises, use the same model as used above.
> 1. Using the code for the generated knowledge approach, score the different answers to the question *without* any additional knowledge. Compare your results to the result of generated knowledge prompting. Did it improve the performance of the model?   
> 2. Implement an example of a few-shot chain-of-thought prompt.
> 3. Try to vary the few-shot and the chain-of-thought prompt by introducing mistakes and inconsistencies. Do these mistakes affect the result of your prediction? Feel free to use any example queries of your choice or reuse the examples above.

**Outlook**

As always, here are a few optional resources on this topic to llok at (although there is definitely much more online):

* a [prompting webbook](https://www.promptingguide.ai/) providing an overview of various approaches
* a framework / package, LangChain, which provides very useful utilities for more complex schemes like [tree of thought prompting](https://github.com/langchain-ai/langchain/blob/master/cookbook/tree_of_thought.ipynb) (spoiler: we will look closer at this package in future sessions, but you can already take a look if you are curious!)